# Delivery Time Prediction - Project Workflow


This notebook walks through the entire pipeline of the Delivery Time Prediction project, from data cleaning to deploying a prediction system using Streamlit.


## 1. Data Cleaning and Preprocessing

In [ ]:

import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("data/raw_dataset.csv")

# Basic cleaning: drop missing values, parse dates
df.dropna(inplace=True)
df['Order_Date'] = pd.to_datetime(df['Order_Date'])
df['Order_Time'] = pd.to_datetime(df['Order_Time']).dt.time
df['Pickup_Time'] = pd.to_datetime(df['Pickup_Time']).dt.time

# Feature engineering: compute distance, time deltas
# Save processed data
df.to_csv("data/processed_data.csv", index=False)
df.head()


## 2. Model Training and Tracking with MLflow

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import mlflow
import mlflow.sklearn

# Select features and target
X = df[['Agent_Age', 'Agent_Rating']]  # add more relevant features
y = df['Delivery_Time']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Start MLflow experiment
mlflow.start_run()

model = RandomForestRegressor()
model.fit(X_train, y_train)
preds = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
mlflow.log_metric("rmse", rmse)
mlflow.sklearn.log_model(model, "rf_model")

mlflow.end_run()
print(f"Model trained. RMSE: {rmse:.2f}")


## 3. Exploratory Data Analysis - Key Insights

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# Delivery Time Distribution
sns.histplot(df['Delivery_Time'], kde=True)
plt.title("Delivery Time Distribution")
plt.show()

# Relationship between rating and delivery time
sns.scatterplot(data=df, x='Agent_Rating', y='Delivery_Time')
plt.title("Agent Rating vs Delivery Time")
plt.show()


## 4. Streamlit Deployment


A Streamlit interface is provided in `app/streamlit_app.py` for user-friendly delivery time prediction.
Run the app using:

```bash
streamlit run app/streamlit_app.py
```
